In [1]:
import pandas as pd
import numpy as np
import random
import time
import sys
import os

sys.path.append(os.path.join("..", ".."))

# Sentence Embedding
from transformers import CanineTokenizer, CanineModel


# Set of the random seed for the kmeans model
from functions.Experimentations import ParamSearch
random.seed(0)

c:\Users\dra98\OneDrive\Documentos\Trabajo\Doctorado\Codigo\myvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Variables 

In [2]:
from hydra import initialize, compose

with initialize(version_base=None, config_path="../../conf"):
    cfg=compose(config_name="main.yaml")

# Model

In [3]:
# model = CanineModel.from_pretrained('google/canine-s')
# tokenizer = CanineTokenizer.from_pretrained('google/canine-s')

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


# Data Import

In [7]:
# Data import
# Relative Path of the dataset, change for your dataset

# dataset_name = "cpn120"
# # Options are "cpn27" and "cpn120"

# type_standardization = "normalize" 
# # options are "raw", "normalize", "normalize_wo_stop", and "lemmatize"

# # import of the data
# data = pd.read_csv(cfg.path[type_standardization][dataset_name], delimiter=",")
# data = data.fillna(value='')

,Concepto,Respuesta,Codigo
0,granito,tierra,terrestre
1,granito,rugoso,texturas
2,granito,construcción,material_construccion
3,granito,desagradable al tacto,texturas
4,granito,raspa,lastimar
...,...,...,...
31859,aptitud,inteligencia,inteligencia
31860,aptitud,condicion necesario,necesario
31861,aptitud,caracter,personalidad
31862,aptitud,personalidad,personalidad


In [8]:
# # Timer
# start = time.time()

# descriptions_matrix = np.zeros( # creatign an empty matrix
#     (
#         len(data.iloc[:,1]),          # the number of data points
#         vector_length       # the number of components of the word embedding
#     )
# )

# for i,description in enumerate(data.iloc[:,1]):
#     vector = model(**tokenizer(description, padding="max_length", truncation=True, return_tensors="pt")).pooler_output.detach().numpy()[0]
#     descriptions_matrix[i,] = vector
# # Concatenate the matrix with the data of each observation

# data_matrix = np.concatenate([descriptions_matrix,data.iloc[:,:]], axis=1)

# end = time.time()
# print("Execution time:", end-start)

# pd.DataFrame(data_matrix).to_csv("{}_canine_raw.csv".format(dataset_name))

Execution time: 113407.16197371483


In [3]:
# Abrir csv con vectores numericos.

dataset_name = "cpn27"
# Options are "cpn27" and "cpn120"

type_standardization = "lemmatize" 
# options are "raw", "normalize", "normalize_wo_stop", and "lemmatize"

if type_standardization == "lemmatize":
    data_matrix = pd.read_csv(cfg.path_embedding_canine[dataset_name][type_standardization], delimiter=',',index_col="Unnamed: 0").to_numpy()
else:
    data_matrix = pd.read_csv(cfg.path_embedding_canine[dataset_name][type_standardization], delimiter=',').to_numpy()


# AC-PLT

In [4]:
from functions.AC_PLT import AC_PLT

parameters = {'n_clusters': np.arange(500, 600, 50)}
ac_plt = AC_PLT()

file_name = cfg.path_results.CANINE+r'{}_{}_{}_results.csv'.format(dataset_name, type_standardization, ac_plt.__class__.__name__)

clf_acplt = ParamSearch(ac_plt, parameters)
clf_acplt.fit(data_matrix[:, :cfg.params.vector_length.sentence_embedding], data_matrix[:, cfg.params.vector_length.sentence_embedding+2],file_name)

c:\Users\dra98\OneDrive\Documentos\Trabajo\Doctorado\Codigo\myvenv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\dra98\OneDrive\Documentos\Trabajo\Doctorado\Codigo\myvenv\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


# Naive Bayes

In [ ]:
from Experimentations import ParamSearch
from sklearn.naive_bayes import GaussianNB

parameters = {'var_smoothing': np.geomspace(1e-06, 1e+02, num=9)}
proc = GaussianNB()

file_name = r'../data/results/canine/lemmatize/{}_{}_results.csv'.format(dataset_name, proc.__class__.__name__)

clf = ParamSearch(proc, parameters)
clf.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2],file_name)

/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated clas

# SVC

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='rbf'
parameters = {'C': np.geomspace(1e-05, 1e+01, num=7)}
svc_rbf = SVC(kernel=kernel, gamma='auto')
fileName = r'../data/results/canine/lemmatize/{}_{}{}_results.csv'.format(dataset_name, svc_rbf.__class__.__name__, kernel)

clf_acplt = ParamSearch(svc_rbf, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated clas

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='linear'
parameters = {'C': np.geomspace(1e-05, 1e+01, num=7)}
svc_linear = SVC(kernel=kernel, gamma='auto')
fileName = r'../data/results/canine/lemmatize/{}_{}{}_results.csv'.format(dataset_name, svc_linear.__class__.__name__, kernel)


clf_acplt = ParamSearch(svc_linear, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated clas

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='poly'
parameters = {'degree': np.arange(1,7)}
svc_poly = SVC(kernel=kernel, gamma='auto')
fileName = r'../data/results/canine/lemmatize/{}_{}{}_results.csv'.format(dataset_name, svc_poly.__class__.__name__, kernel)

clf_acplt = ParamSearch(svc_poly, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated clas

In [ ]:
from Experimentations import ParamSearch
from sklearn.svm import SVC

kernel='sigmoid'
parameters = {'C': np.geomspace(1e-05, 1e+01, num=7)}
svc_sigmoid = SVC(kernel=kernel, gamma='auto')
fileName = r'../data/results/canine/lemmatize/{}_{}{}_results.csv'.format(dataset_name, svc_rbf.__class__.__name__, kernel)

clf_acplt = ParamSearch(svc_sigmoid, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated clas

# Decision Tree

In [ ]:
from Experimentations import ParamSearch
from sklearn.tree import DecisionTreeClassifier

criterion = 'gini'
parameters = {'max_leaf_nodes': np.arange(100, 1501, 100)}
desition_tree = DecisionTreeClassifier(criterion=criterion)
fileName = r'../data/results/canine/lemmatize/{}_{}_{}_results.csv'.format(dataset_name, desition_tree.__class__.__name__, criterion)

clf_acplt = ParamSearch(desition_tree, parameters)
clf_acplt.fit(data_matrix[:, :vector_length], data_matrix[:, vector_length+2], fileName)

/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated clas

# XGBoost

In [ ]:
df = pd.DataFrame(data_matrix)
cod = df.iloc[:,vector_length+2].value_counts()
reduce_cod = cod[cod<5]
n=5
data_fill = df.copy()

for key, value in reduce_cod.items():
    m=np.abs(n-value)
    nrow = np.zeros(vector_length)
    nrow = np.concatenate([nrow, np.array(['', '', key])])
    for i in range(m): data_fill = np.vstack([data_fill,nrow])

In [ ]:
y = data_fill[:, vector_length+2]

labels = np.unique(y)
i=0
idx2class = {}
class2idx= {}
for tp in labels:
    idx2class[i] = tp
    class2idx[tp] = i
    i += 1

y_label = np.vectorize(class2idx.get)(y)

In [ ]:
from xgboost import XGBClassifier
from Experimentations import ParamSearch

max_levels = [5,10,50,100,200,300]

parameters = {'max_depht': max_levels, 'n_estimators': np.arange(1, 20, 1)}

bst = XGBClassifier(learning_rate=1, objective='multi:softprob', random_state=0)
fileName = r'../data/results/canine/lemmatize/{}_{}_results.csv'.format(dataset_name, bst.__class__.__name__)

clf_acplt = ParamSearch(bst, parameters)
clf_acplt.fit(data_fill[:, :vector_length], y_label, fileName)


/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:06:33] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depht" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:19:11] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depht" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:32:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depht" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:45:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "max_depht" } are not used.

  warnings.warn(smsg, UserWarning)
/home/jsmf/Desktop/DiegoRamos/.env/lib/python3.10/site-packages/xgboost/core.py:160: Use

# Random Forest
Ejecutar las celdas de Xgboost

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from Experimentations import ParamSearch

max_levels = [5,10,50,100,200,300]

parameters = {'max_depht': max_levels}
rndforest = RandomForestClassifier(random_state=0)
fileName = r'../data/results/canine/lemmatize/{}_{}_results.csv'.format(dataset_name, rndforest.__class__.__name__)

clf_acplt = ParamSearch(rndforest, parameters)
clf_acplt.fit(data_fill[:, :vector_length], y_label, fileName)


# KNN

In [ ]:
from Experimentations import ParamSearch
from sklearn.neighbors import KNeighborsClassifier

parameters = {'n_neighbors': np.arange(10, 501, 10)}
knn = KNeighborsClassifier()
fileName = r'../data/results/canine/lemmatize/{}_{}_results.csv'.format(dataset_name, knn.__class__.__name__)

clf_acplt = ParamSearch(knn, parameters)
clf_acplt.fit(data.iloc[:, :vector_length].to_numpy(), data.iloc[:, vector_length+2].to_numpy(), fileName)